In [1]:
### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [2]:
!pip install feature-engine
from feature_engine.encoding import WoEEncoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 1.5 MB/s eta 0:00:00a 0:00:01


In [3]:
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression, HuberRegressor
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

import pickle

In [4]:
train = pd.read_csv('../input/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2022/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-aug-2022/sample_submission.csv')

print("train", train.shape)
print("test", test.shape)
display(train.head())
display(test.head())

train (26570, 26)
test (20775, 25)


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044


In [5]:
train.isna().sum()

id                   0
product_code         0
loading            250
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      381
measurement_4      538
measurement_5      676
measurement_6      796
measurement_7      937
measurement_8     1048
measurement_9     1227
measurement_10    1300
measurement_11    1468
measurement_12    1601
measurement_13    1774
measurement_14    1874
measurement_15    2009
measurement_16    2110
measurement_17    2284
failure              0
dtype: int64

In [6]:
test.isna().sum()

id                   0
product_code         0
loading            223
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      329
measurement_4      409
measurement_5      508
measurement_6      624
measurement_7      720
measurement_8      846
measurement_9      904
measurement_10    1067
measurement_11    1136
measurement_12    1240
measurement_13    1303
measurement_14    1440
measurement_15    1542
measurement_16    1678
measurement_17    1740
dtype: int64

In [7]:
measurement_cols = []
attribute_cols = []
for i in train.columns:
    if "measurement" in i:
        measurement_cols.append(i)
    elif "attribute" in i:
        attribute_cols.append(i)
        
feat_object = train.select_dtypes(np.object).columns
        
print(measurement_cols)
print(attribute_cols)
print()
print(feat_object)

['measurement_0', 'measurement_1', 'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12', 'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16', 'measurement_17']
['attribute_0', 'attribute_1', 'attribute_2', 'attribute_3']

Index(['product_code', 'attribute_0', 'attribute_1'], dtype='object')


In [8]:
ep = 1.9
neighbor = 3

def preprocessing(data):
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)

    # highly correlative features 
    fill_dict = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    feature = measurement_cols.copy()
    feature.append('loading')

    for code in data.product_code.unique(): # ABCDEFGHI
        tmp = data[data.product_code == code]
        column = fill_dict[code]
        tmp_train = tmp[column + ['measurement_17']].dropna(how='any')
        tmp_test = tmp[(tmp[column].isnull().sum(axis=1) == 0) & (tmp['measurement_17'].isnull())]

        regressor = HuberRegressor(epsilon=ep)
        regressor.fit(tmp_train[column], tmp_train['measurement_17'])
        data.loc[(data.product_code == code) & (data[column].isnull().sum(axis=1) == 0) 
                 & (data['measurement_17'].isnull()), 'measurement_17'] = regressor.predict(tmp_test[column])

        imputer = KNNImputer(n_neighbors=neighbor)
        data.loc[data.product_code == code, feature] = imputer.fit_transform(data.loc[data.product_code == code, feature])
        print(code, ":", len(tmp_test), "null samples imputed")

In [9]:
select_feats = [
    'loading',
    'attribute_0',
    'measurement_0',
    'measurement_1',
    'measurement_2',
    'measurement_17',
    'm3_missing', 
    'm5_missing'
]

# Select
def select(train_df, test_df):
    train_select = train_df[select_feats]
    test_select = test_df[select_feats]
    
#     display(train_select.head())
#     display(test_select.head())
    
    return train_select, test_select

In [10]:
# Encode
def encode(train_df, result_df, test_df):
    woe_encoder = WoEEncoder(variables=['attribute_0'])
    woe_encoder.fit(train_df, result_df)
    train_encoded = woe_encoder.transform(train_df)
    test_encoded = woe_encoder.transform(test_df)

#     display(train_encoded.head())
#     display(test_encoded.head())

    return train_encoded, test_encoded

In [11]:
# Scale
def scale(train_df, test_df):
    scaler = StandardScaler()

    train_scaled = train_df
    test_scaled = test_df

    train_scaled[select_feats] = scaler.fit_transform(train_df)
    test_scaled[select_feats] = scaler.transform(test_df)

#     display(train_scaled.head())

    return train_scaled, test_scaled

In [12]:
# Concatenate df's
data = pd.concat([train, test])
print(data.shape)

preprocessing(data)

train_processed = data[data.failure.notnull()].copy()
test_processed = data[data.failure.isnull()].copy()
test_processed = test_processed.drop(['failure'], axis=1)

# display(train_processed.head())

train_factor = train_processed.drop(['failure'], axis=1)
train_result = train_processed['failure'].astype(int)

train_select, test_select = select(train_factor, test_processed)

train_encoded, test_encoded = encode(train_select, train_result, test_select)

train_scaled, test_scaled = scale(train_encoded, test_encoded)

(47345, 26)
A : 386 null samples imputed
B : 418 null samples imputed
C : 391 null samples imputed
D : 398 null samples imputed
E : 429 null samples imputed
F : 420 null samples imputed
G : 373 null samples imputed
H : 361 null samples imputed
I : 377 null samples imputed


In [13]:
# !rm model_skf.sav model_Nkf.sav

rm: cannot remove 'model_skf.sav': No such file or directory
rm: cannot remove 'model_Nkf.sav': No such file or directory


# Using KFOLD

In [14]:
# Train
y_pred = np.zeros(len(test))

auc = 0
avg_auc = 0

best_auc = 0

splits = 5
skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=0)

for i, (train_idx, val_idx) in enumerate(skf.split(train_scaled, train_result)):
    x_train, x_val = train_scaled.iloc[train_idx], train_scaled.iloc[val_idx]
    y_train, y_val = train_result.iloc[train_idx], train_result.iloc[val_idx]
    
    model = LogisticRegression(penalty='l1', solver='liblinear', C=1e-2, class_weight='balanced', max_iter=200, tol=1e-4, n_jobs=-1, random_state=123)
    model.fit(x_train, y_train)

    val_pred = model.predict_proba(x_val)
    auc = roc_auc_score(y_val, val_pred[:,1])
    
    if (auc > best_auc):
        best_model = model
    
    avg_auc += auc
    print("Fold:", i + 1, "auc:", round(auc, 4))
    
#     y_pred += model.predict_proba(test_scaled)[:,1]

avg_auc /= splits
# y_pred /= splits

print("Average auc:", round(avg_auc, 4))

Fold: 1 auc: 0.6013
Fold: 2 auc: 0.5922
Fold: 3 auc: 0.5829
Fold: 4 auc: 0.5939
Fold: 5 auc: 0.5855
Average auc: 0.5912


In [15]:
filename = 'model_skf.sav'
pickle.dump(best_model, open(filename, 'wb'))

# Not Using KFOLD

In [16]:
# not using KFold

y_pred = np.zeros(len(test))
X_train, X_test , y_train , y_test = train_test_split(train_scaled, train_result, test_size=0.2, random_state=123)

model = LogisticRegression(penalty='l1', solver='liblinear', C=1e-2, class_weight='balanced',max_iter=200, tol=1e-4, n_jobs=-1, random_state=123)

model.fit(X_train[select_feats], y_train)

LogisticRegression(C=0.01, class_weight='balanced', max_iter=200, n_jobs=-1,
                   penalty='l1', random_state=123, solver='liblinear')

In [17]:
filename = 'model_Nkf.sav'
pickle.dump(model, open(filename, 'wb'))